# Yolo v8 segmentation 모델 평가
cozy house 앱 서비스의 상황을 잘 표현한 데이터셋으로 모델을 평가하고 결과 확인
- '사람이 문앞에 20초 이상 머무름 -> 위험 알림' task에 적합한 모델인지 확인
  - 추가 모델 학습이 필요한지 확인 -> 성능이 좋지 않을경우 추가 데이터셋을 만들고, training + test
  - 성능이 좋게 나올경우 : pose estimation 모델로 추가 task 구현 시도

# Mission : video 데이터를 single frame형태로 변환

## Try1 : model.val()
- 파라미터없이 모델 자체의 성능 확인

In [3]:
# !pip install ultralytics

     |████████████████████████████████| 722 kB 6.6 MB/s            
     |████████████████████████████████| 78 kB 10.1 MB/s            
     |████████████████████████████████| 62.2 MB 15 kB/s              
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.3
    Uninstalling tqdm-4.62.3:
      Successfully uninstalled tqdm-4.62.3
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.5.3.56
    Uninstalling opencv-python-4.5.3.56:
      Successfully uninstalled opencv-python-4.5.3.56


In [7]:
from ultralytics import YOLO

# Load the YOLOv8 segmentation model
model = YOLO('yolov8n-seg.pt')

# validate model
validation_results = model.val(data='coco8.yaml', 
                               imgsz=640,
                               batch=16,
                               conf=0.25,
                               iou=0.6,
                               device='mps')

print(f"Mean Average Precision @.5:.95 : {metrics.box.map}")    
print(f"Mean Average Precision @ .50   : {metrics.box.map50}") 
print(f"Mean Average Precision @ .70   : {metrics.box.map75}")

Ultralytics YOLOv8.1.30 🚀 Python-3.9.7 torch-1.9.1+cu111 CUDA:0 (Tesla T4, 15110MiB)
YOLOv8n-seg summary (fused): 195 layers, 3404320 parameters, 0 gradients, 12.6 GFLOPs


val: Scanning /aiffel/aiffel/Aiffelthon/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/8 [00:00<?, ?it/s]


IndexError: The shape of the mask [100] at index 0 does not match the shape of the indexed tensor [0, 1] at index 0

- 이전에 작업했던 데이터셋 경로로 테스트해서 오류가 발생하는듯함
- 사용하지않았던 데이터셋으로 다운받아서 진행

In [8]:
# validate model
validation_results = model.val(data='coco.yaml')

print(f"Mean Average Precision @.5:.95 : {metrics.box.map}")    
print(f"Mean Average Precision @ .50   : {metrics.box.map50}") 
print(f"Mean Average Precision @ .70   : {metrics.box.map75}")

Ultralytics YOLOv8.1.30 🚀 Python-3.9.7 torch-1.9.1+cu111 CUDA:0 (Tesla T4, 15110MiB)

Dataset 'coco.yaml' images not found ⚠️, missing path '/aiffel/aiffel/Aiffelthon/datasets/coco/val2017.txt'


  0%|          | 0.00/169M [00:00<?, ?B/s]

Unzipping /aiffel/aiffel/Aiffelthon/datasets/coco2017labels-segments.zip to /aiffel/aiffel/Aiffelthon/datasets/coco...: 100%|██████████| 122232/122232 [00:19<00:00, 6371.05file/s]

KeyboardInterrupt: 

- 너무 느리고 진행상황을 알수 없어서 중단

## Try 2: 자체 제작 데이터셋의 yaml파일을 만들어서 검증
- [roboflow](https://roboflow.com/)에서 yolo v8 형태로 데이터셋 제작
- [ultralytics guide](https://www.youtube.com/watch?v=LNwODJXcvt4&t=29s) 참고

In [10]:
# !pip install roboflow
# 이 부분은 roboflow에서 code snipet 사용
from roboflow import Roboflow
rf = Roboflow(api_key="private_api_key")
project = rf.workspace("cozy-house").project("cozy-house")
version = project.version(1)
dataset = version.download("yolov8")

     |████████████████████████████████| 71 kB 1.1 MB/s             
     |████████████████████████████████| 54 kB 540 kB/s             
     |████████████████████████████████| 49.1 MB 28 kB/s              
     |████████████████████████████████| 158 kB 44.5 MB/s            
     |████████████████████████████████| 178 kB 4.3 MB/s            
  Attempting uninstall: certifi
    Found existing installation: certifi 2021.10.8
    Uninstalling certifi-2021.10.8:
      Successfully uninstalled certifi-2021.10.8
  Attempting uninstall: cycler
    Found existing installation: cycler 0.11.0
    Uninstalling cycler-0.11.0:
      Successfully uninstalled cycler-0.11.0
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.5.4.60
    Uninstalling opencv-python-headless-4.5.4.60:
      Successfully uninstalled opencv-python-headless-4.5.4.60
  Attempting uninstall: chardet
    Found existing installation: chardet 3.0.4
    Uninstalling chardet-3.0.4


Extracting Dataset Version Zip to cozy-house-1 in yolov8:: 100%|██████████| 110/110 [00:00<00:00, 2508.09it/s]


In [12]:
!yolo task=detect mode=val model=yolov8n-seg.pt data={dataset.location}/data.yaml

WARNING ⚠️ conflicting 'task=detect' passed with 'task=segment' model. Ignoring 'task=detect' and updating to 'task=segment' to match model.
Ultralytics YOLOv8.1.30 🚀 Python-3.9.7 torch-1.9.1+cu111 CPU (Intel Xeon 2.20GHz)
YOLOv8n-seg summary (fused): 195 layers, 3404320 parameters, 0 gradients, 12.6 GFLOPs
Traceback (most recent call last):
  File "/opt/conda/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/opt/conda/lib/python3.9/site-packages/ultralytics/cfg/__init__.py", line 568, in entrypoint
    getattr(model, mode)(**overrides)  # default args from model
  File "/opt/conda/lib/python3.9/site-packages/ultralytics/engine/model.py", line 514, in val
    validator(model=self.model)
  File "/opt/conda/lib/python3.9/site-packages/torch/autograd/grad_mode.py", line 28, in decorate_context
    return func(*args, **kwargs)
  File "/opt/conda/lib/python3.9/site-packages/ultralytics/engine/validator.py", line 143, in __call__
    self.data = check_det_dataset(self.args.d

- 경로세팅 오류 -> 해결하고 다시 시도

In [21]:
!yolo task=detect mode=val model=yolov8n.pt data={dataset.location}/data.yaml device=cuda

Ultralytics YOLOv8.1.30 🚀 Python-3.9.7 torch-1.9.1+cu111 CUDA:0 (Tesla T4, 15110MiB)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs
val: Scanning /aiffel/aiffel/Aiffelthon/cozy-house-1/valid/labels... 52 images, 
val: New cache created: /aiffel/aiffel/Aiffelthon/cozy-house-1/valid/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all         52         52          1      0.998      0.995      0.963
                person         52         52          1      0.998      0.995      0.963
Speed: 9.2ms preprocess, 23.2ms inference, 0.0ms loss, 6.7ms postprocess per image
Results saved to runs/detect/val9
💡 Learn more at https://docs.ultralytics.com/modes/val


- 나머지 데이터셋에 대해서도 검증진행
    - 비슷한 결과가 나오는지 확인

In [22]:
# IMG_4970, IMG_4971 dataset

rf = Roboflow(api_key="my_app_key")
project = rf.workspace("cozy-house").project("cozy-house")
version = project.version(2)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.30, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to cozy-house-2 in yolov8:: 100%|██████████| 1446/1446 [00:01<00:00, 1153.73it/s]


In [23]:
# dataset 주소 변경 - 2번째 테스트
!yolo task=detect mode=val model=yolov8n.pt data={dataset.location}/data.yaml device=cuda

Ultralytics YOLOv8.1.30 🚀 Python-3.9.7 torch-1.9.1+cu111 CUDA:0 (Tesla T4, 15110MiB)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs
val: Scanning /aiffel/aiffel/Aiffelthon/cozy-house-2/valid/labels... 720 images,
val: New cache created: /aiffel/aiffel/Aiffelthon/cozy-house-2/valid/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        720        720      0.998      0.881      0.929      0.895
                person        720        720      0.998      0.881      0.929      0.895
Speed: 0.7ms preprocess, 5.7ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/val10
💡 Learn more at https://docs.ultralytics.com/modes/val


- 비슷한 성능을 보임
- 우리 데이터셋에만 좋은 성능이 나오는건지 확인차 AI 허브 데이터셋으로 테스트

In [24]:
# AI hub dataset : VS12_도어락터치 
# - C021_A18_SY17_P07_S06_01DBS.mp4
# - C021_A18_SY17_P07_S07_02DBS.mp4

# from roboflow import Roboflow
rf = Roboflow(api_key="my_app_key")
project = rf.workspace("cozy-house").project("cozy-house")
version = project.version(3)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.30, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to cozy-house-3 in yolov8:: 100%|██████████| 9124/9124 [00:03<00:00, 2748.51it/s]


In [25]:
!yolo task=detect mode=val model=yolov8n.pt data={dataset.location}/data.yaml device=cuda

Ultralytics YOLOv8.1.30 🚀 Python-3.9.7 torch-1.9.1+cu111 CUDA:0 (Tesla T4, 15110MiB)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs
val: Scanning /aiffel/aiffel/Aiffelthon/cozy-house-3/valid/labels... 4559 images
val: New cache created: /aiffel/aiffel/Aiffelthon/cozy-house-3/valid/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all       4559       4559          1      0.981      0.993      0.961
                person       4559       4559          1      0.981      0.993      0.961
/opt/conda/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Speed: 0.3ms preprocess, 3.9ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/val11
💡 Learn more at https://docs.ultralytics.com/m